In [1]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
from dotenv import load_dotenv
from langchain_pinecone import PineconeVectorStore
import os

from sympy.physics.units import temperature


In [2]:
loader=PyPDFLoader('yolov9_paper.pdf')
data=loader.load()

In [3]:
type(data)

list

In [6]:
len(data)

18

In [7]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000)
docs=text_splitter.split_documents(data)
print(len(docs))

96


In [8]:
docs[90]

Document(metadata={'source': 'yolov9_paper.pdf', 'page': 16}, page_content='RT DETR-R101 [43] (I) 76 259 54.3 72.7 58.6 36.0 58.8 72.1\nRTMDet-X [44] (I) 94.9 283.4 52.8 70.4 – – – –\nYOLOR-CSP-X [66] (C) 96.9 226.8 54.8 73.1 59.7 – – –\nPPYOLOE+-X [74] (C) 98.4 206.6 54.7 72.0 59.9 37.9 59.3 70.4\nPPYOLOE-X [74] (I) 98.4 206.6 52.3 69.5 56.8 35.1 57.0 68.6\n1 (S), (I), (D), (C) indicate train-from-scratch, ImageNet pretrained, knowledge distillation, and complex setting, respectively.\nTable 4 shows the performance of all models sorted by\nparameter size. Our proposed YOLOv9 is Pareto optimal\nin all models of different sizes. Among them, we found no\nother method for Pareto optimal in models with more than\n20M parameters. The above experimental data shows that\nour YOLOv9 has excellent parameter usage efficiency.\nShown in Table 5 is the performance of all participat-\ning models sorted by the amount of computation. Our pro-\nposed YOLOv9 is Pareto optimal in all models with differ-

In [9]:
embeddings=GoogleGenerativeAIEmbeddings(model='models/embedding-001',api_key=os.getenv('GOOGLE_API_KEY'))
embeddings

GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000022B463E8A10>, model='models/embedding-001', task_type=None, google_api_key=SecretStr('**********'), credentials=None, client_options=None, transport=None, request_options=None)

In [10]:
vector=embeddings.embed_query('heyy')
vector

[0.03700842708349228,
 -0.01657170243561268,
 -0.045119449496269226,
 -0.015112409368157387,
 0.022411750629544258,
 -0.023157892748713493,
 0.03575361147522926,
 0.0024629919789731503,
 0.0039060628041625023,
 -0.00043142109643667936,
 0.017006056383252144,
 0.04717766121029854,
 -0.03150999918580055,
 -0.04243204742670059,
 0.03548401594161987,
 -0.010295159183442593,
 -0.005288803018629551,
 -6.547382508870214e-05,
 -0.005637001246213913,
 -0.03341652452945709,
 0.01899448223412037,
 0.028171060606837273,
 -0.016946090385317802,
 0.005532759707421064,
 0.03745630756020546,
 -0.01768657937645912,
 0.028783218935132027,
 -0.01719924621284008,
 -0.024988917633891106,
 0.02237369306385517,
 -0.02618817612528801,
 0.03764578327536583,
 -0.04485812038183212,
 0.01920115016400814,
 0.02374928817152977,
 -0.06173095107078552,
 0.043428272008895874,
 0.00743491854518652,
 -0.012311708182096481,
 0.024208582937717438,
 0.00862980354577303,
 -0.08366993069648743,
 -0.016559571027755737,
 -0.01

In [11]:
len(vector)

768

In [12]:
PINECONE_INDEX_NAME='sample'
os.environ['PINECONE_API_KEY']=('pcsk_6yjLVo_3pw6MjXiZx4VEcg6UycLcaHQHQod4BaHCb7pZnJrjxZdzUmNAiuWF6pqu9bJnA7')
docsearch=Pinecone.from_existing_index(index_name=PINECONE_INDEX_NAME,embedding=embeddings)
print("Index successfully created")

Index successfully created


In [13]:
vectorstore_from_docs=PineconeVectorStore.from_documents(docs,index_name=PINECONE_INDEX_NAME,embedding=embeddings)

In [14]:
docsearch=PineconeVectorStore.from_existing_index(PINECONE_INDEX_NAME,embeddings)

In [15]:
query= "what is parameters"
docs=docsearch.similarity_search(query,k=3)
print(docs)

[Document(id='9e4438b0-1e27-41db-b336-7f464393a3ac', metadata={'page': 1.0, 'source': 'yolov9_paper.pdf'}, page_content='ditional layers to combine repeatedly fed input data, which\nwill significantly increase the inference cost. In addition,\nsince the input data layer to the output layer cannot have a\ntoo deep path, this limitation will make it difficult to model\nhigh-order semantic information during the training pro-\ncess. As for masked modeling, its reconstruction loss some-\ntimes conflicts with the target loss. In addition, most mask\nmechanisms also produce incorrect associations with data.\nFor the deep supervision mechanism, it will produce error\naccumulation, and if the shallow supervision loses informa-\ntion during the training process, the subsequent layers will\nnot be able to retrieve the required information. The above\nphenomenon will be more significant on difficult tasks and\nsmall models.\nTo address the above-mentioned issues, we propose a\nnew concept, which 

In [16]:
retriever=docsearch.as_retriever(search_type='similarity',search_kwargs={'k':10})
retrieved_docs=retriever.invoke("What is yolov9 Parameter")

In [17]:
len(retrieved_docs)

10

In [18]:
print(retrieved_docs[5].page_content)

RT DETR-R101 [43] (I) 76 259 54.3 72.7 58.6 36.0 58.8 72.1
RTMDet-X [44] (I) 94.9 283.4 52.8 70.4 – – – –
YOLOR-CSP-X [66] (C) 96.9 226.8 54.8 73.1 59.7 – – –
PPYOLOE+-X [74] (C) 98.4 206.6 54.7 72.0 59.9 37.9 59.3 70.4
PPYOLOE-X [74] (I) 98.4 206.6 52.3 69.5 56.8 35.1 57.0 68.6
1 (S), (I), (D), (C) indicate train-from-scratch, ImageNet pretrained, knowledge distillation, and complex setting, respectively.
Table 4 shows the performance of all models sorted by
parameter size. Our proposed YOLOv9 is Pareto optimal
in all models of different sizes. Among them, we found no
other method for Pareto optimal in models with more than
20M parameters. The above experimental data shows that
our YOLOv9 has excellent parameter usage efficiency.
Shown in Table 5 is the performance of all participat-
ing models sorted by the amount of computation. Our pro-
posed YOLOv9 is Pareto optimal in all models with differ-
ent scales. Among models with more than 60 GFLOPs, only


In [19]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm=ChatGoogleGenerativeAI(model='gemini-1.5-pro',temperature=0.3,max_tokens=500)

In [26]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt=(
    "You are an assistant for question-answering tasks Use the following pieces of retrieved context to answer The question. If You dont know the answer ,say that you dont know. Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)
prompt=ChatPromptTemplate.from_messages([("system",system_prompt),("human","{input}"),])

In [27]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [28]:
response=rag_chain.invoke({"input":"YOLOv9 Parameter? "})
print(response["answer"])


The training parameters for YOLOv9 include 500 epochs, an SGD optimizer, a linear learning rate decay from 0.01 to 0.0001, and a momentum of 0.937.  The weight decay is 0.0005, and warm-up epochs are set to 3.  Various data augmentation settings are also used, such as HSV saturation and value augmentation, translation, scale, mosaic, MixUp, and copy & paste.



In [29]:
response=rag_chain.invoke({"input":"YOLOv9 architecture change?  "})
print(response["answer"])

YOLOv9 replaces the ELAN (Extended Efficient Layer Aggregation Network) block in YOLOv7 with a new block called GELAN (General ELAN) and uses CSP-ELAN as the component unit of GELAN.  The downsampling module is also simplified, and the anchor-free prediction head is optimized using a decoupled branch.



In [30]:
response=rag_chain.invoke({"input":"what is medical science "})
print(response["answer"])

This question cannot be answered from the given context.  The provided text discusses deep learning and neural networks, focusing on computer vision applications like object detection. It does not define or explain medical science.

